In [2]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

In [3]:
epochs = 10
range_min = -999
range_max = 999
batch_size = 4 
num_classes = 2
num_train_samples = 1000
num_test_samples = 100

In [8]:
x_train = np.random.randint(low = range_min,high = range_max+1,size = (num_train_samples,1))
y_train = np.abs(x_train)
y_train.shape

(1000, 1)

In [9]:
x_test = np.random.randint(low = range_min,high = range_max+1,size = (num_test_samples,1))
y_test = np.abs(x_test)
y_test.shape

(100, 1)

In [10]:
train_data = torch.from_numpy(x_train).type(torch.FloatTensor)
train_labels = torch.from_numpy(y_train).type(torch.LongTensor)
test_data = torch.from_numpy(x_test).type(torch.FloatTensor)
test_labels = torch.from_numpy(y_test).type(torch.LongTensor)

In [11]:
# This model adds two numbers
class Net_add(nn.Module):
    def __init__(self):
        super(Net_add,self).__init__()
        self.fc1 = nn.Linear(2,1,bias=False)
        self.fc1.weight.data = torch.tensor([[1,1]]).type(torch.FloatTensor)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)

model_add = torch.load('model_add')

In [12]:
# This model subtracts two numbers
class Net_sub(nn.Module):
    def __init__(self):
        super(Net_sub,self).__init__()
        self.fc1 = nn.Linear(2,1)
        self.fc1.weight.data = torch.tensor([[1,-1]]).type(torch.FloatTensor)
        self.fc1.bias.data = torch.tensor([0]).type(torch.FloatTensor)
    
    def forward(self,x):
        x = self.fc1(x)
        return x
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)
    
model_sub = torch.load('model_sub')

In [13]:
# This model will give the absolute of a number
# input: 1 number
# output: absolute of that number 

class Net_abs(nn.Module):
    def __init__(self,model_add,model_sub):
        super(Net_abs,self).__init__()
        
        self.model_add = model_add
        self.model_sub = model_sub
        self.maxpool1d = nn.MaxPool1d(2)

    
    def forward(self,x):
        concat = torch.cat((x,x),1)
        add = self.model_add.predict(concat)
        sub = self.model_sub.predict(concat)
        
        concat = torch.cat((add,sub),1).unsqueeze(0)
        out = self.maxpool1d(concat).squeeze(0)
        
        concat = torch.cat((out,x),1)
        out = model_sub.predict(concat)
        
        return out
    
    def predict(self,x):
        return self.forward(x).type(torch.FloatTensor)

In [14]:
net = Net_abs(model_add,model_sub)

In [17]:
# print(train_data[0:10].shape)
print(train_labels[0:10])
net.forward(train_data[0:10])

tensor([[869],
        [700],
        [414],
        [492],
        [ 69],
        [652],
        [695],
        [236],
        [313],
        [647]])


tensor([[869.],
        [700.],
        [414.],
        [492.],
        [ 69.],
        [652.],
        [695.],
        [236.],
        [313.],
        [647.]], grad_fn=<AddmmBackward>)

In [18]:
torch.save(net,'model_abs')

C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_abs. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_add. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\Ammar\.conda\envs\neuralnets\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Net_sub. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
